## Hi this notebook is for a Coursera assigment ^_^

In [1]:
import numpy as np
import pandas as pd


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Now I'll start week three projct 
### Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [3]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head(12)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
df.groupby('Borough').nunique()

,Postcode,Borough,Neighbourhood
Borough,,,
Central Toronto,9,1,17
Downtown Toronto,18,1,36
East Toronto,5,1,7
East York,5,1,6
Etobicoke,12,1,45
Mississauga,1,1,1
North York,24,1,38
Not assigned,77,1,1
Queen's Park,1,1,1


In [5]:
# remove any Not assigned Borough, 
df2 = df[df['Borough'] != "Not assigned"].reset_index(drop=True)
# group all Borough with the same Postcode
df3 = df2.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
# if Neighbourhood is Not assigned then make it = to Borough
for index, row in df3.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
# show the number of rows,culms
df3.shape 

(103, 3)